# <span style="color:Green">Load the DataSet (Dataset is large, so can take some time)</span>

In [7]:
import pandas as pd

# Load the dataset
df = pd.read_csv('35100177.csv', low_memory=False) # It was given warning as there is not a particular datatype's dataset, so used low_memory=False
print(df)

int64

Updated DataFrame with Year as integer:
int32
         REF_DATE                                      GEO           DGUID  \
0            1998                                   Canada  2021A000011124   
1            1998                                   Canada  2021A000011124   
2            1998                                   Canada  2021A000011124   
3            1998                                   Canada  2021A000011124   
4            1998                                   Canada  2021A000011124   
...           ...                                      ...             ...   
7244905      2023  Canadian Forces Military Police [99001]             NaN   
7244906      2023  Canadian Forces Military Police [99001]             NaN   
7244907      2023  Canadian Forces Military Police [99001]             NaN   
7244908      2023  Canadian Forces Military Police [99001]             NaN   
7244909      2023  Canadian Forces Military Police [99001]             NaN   

          

# <span style="color:Green">In raw DataSet, there is too many rows, so we transpose rows to columns, with Data Cleaning</span>

In [2]:
# Cleaning missing values
df['VALUE'] = pd.to_numeric(df['VALUE'], errors='coerce')

# Ensure no duplicates in the pivot key
assert not df.duplicated(subset=['REF_DATE', 'GEO', 'Violations', 'Statistics']).any(), "Duplicate entries found!"

# Pivot the table
df = df.pivot_table(
    index=['REF_DATE', 'GEO', 'Violations'],
    columns='Statistics',
    values='VALUE',
    aggfunc='first'  # Use 'first' only if there is no duplicates 
).reset_index()

# Clean columns names
df.columns = [col.replace(' ', '_').replace(',', '') for col in df.columns]

# <span style="color:Green">Removing columns which has more than 50% missing values</span>

In [3]:
# getting the number of missing values by column wise
missing_counts = df.isnull().sum()

# getting the percentage of missing values column wise
missing_percentage = (missing_counts / len(df)) * 100

# Printing the missing value count with percentages
missing_info = pd.DataFrame({
    'Missing Values': missing_counts,
    'In %': missing_percentage
})
print(missing_info)




# remove columns which has more than 50% missing value
threshold = len(df) * 0.5
# print(threshold)

df = df.dropna(axis=1, thresh=threshold)

# again getting the number of missing values by column wise, after removing columns which has more than 50% missing value
print(df.isnull().sum())

                                                    Missing Values       In %
REF_DATE                                                         0   0.000000
GEO                                                              0   0.000000
Violations                                                       0   0.000000
Actual_incidents                                                 0   0.000000
Cleared_by_charge                                                0   0.000000
Cleared_otherwise                                                0   0.000000
Percent_unfounded                                           243648  78.891335
Percentage_change_in_rate                                   181410  58.739153
Percentage_contribution_to_the_Crime_Severity_I...           19155   6.202241
Rate_per_100000_population                                    1395   0.451690
Rate_adult_charged_per_100000_population_aged_1...            1428   0.462375
Rate_total_persons_charged_per_100000_populatio...            14

# <span style="color:Green">Handling Remaining missing values by mean</span>

In [4]:
# Store row count before handling missing values  
initial_rows = len(df)
print("Total rows before:", initial_rows)

# Your key and value columns
key_columns = ['GEO', 'Violations', 'REF_DATE']
value_columns = [col for col in df.columns if col not in key_columns]


# Step 1: If there is missing value in a particular column then it should be mean of particular crime with particular location with all the years
for col in value_columns:
    df[col] = df[col].fillna(df.groupby(['GEO', 'Violations'])[col].transform('mean'))

# Step 2: Still there is missing values i.e there can be missing value in all the years for particular crime with particular location, so we removed that rows from the dataset
for col in value_columns:
    # Find groups where the column is all NaN with consideration of GEO and Violations
    mask = df.groupby(['GEO', 'Violations'])[col].transform(lambda x: x.isna().all())
    df = df[~mask]



# Checking for missing values after handling
print(df.isnull().sum())


# Store row count after handling missing values  
final_rows = len(df)
print("Total rows after:", final_rows)
print("Total rows removed:", initial_rows - final_rows)

Total rows before: 308840
REF_DATE                                                                   0
GEO                                                                        0
Violations                                                                 0
Actual_incidents                                                           0
Cleared_by_charge                                                          0
Cleared_otherwise                                                          0
Percentage_contribution_to_the_Crime_Severity_Index_(CSI)                  0
Rate_per_100000_population                                                 0
Rate_adult_charged_per_100000_population_aged_18_years_and_over            0
Rate_total_persons_charged_per_100000_population_aged_12_years_and_over    0
Rate_youth_charged_per_100000_population_aged_12_to_17_years               0
Rate_youth_not_charged_per_100000_population_aged_12_to_17_years           0
Total_cleared                                     

# <span style="color:Green">Download the Preprocessed File</span>

In [5]:
# It is downloading, so take some time
df.to_csv('preprocessed_crime_data.csv', index=False)